In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import seaborn as sns
%matplotlib inline
#%matplotlib notebook
import nltk
import gensim
# import spacy
from sklearn.svm import SVC
# for accuracy mertic
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv('train.csv')
df.head(10)

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...
5,Tr-6,8/2/2016,Mortgage,"Loan modification,collection,foreclosure",NaN,8/3/2016,Closed with explanation,Yes,The owner of my original mortgage filed for ba...
6,Tr-7,3/26/2017,Credit reporting,Incorrect information on credit report,NaN,3/26/2017,Closed with explanation,No,J'ai été victime d'une fraude d'identité et j'...
7,Tr-8,10/15/2016,Bank account or service,Problems caused by my funds being low,Company has responded to the consumer and the ...,10/15/2016,Closed with explanation,No,"Je suis en train de faire faillite et, par con..."
8,Tr-9,1/18/2016,Debt collection,Cont'd attempts collect debt not owed,NaN,1/18/2016,Closed with explanation,Yes,Una agencia de cobranza me hizo adulterar de q...
9,Tr-10,8/17/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,8/17/2015,Closed with non-monetary relief,No,"Le XXXX / XXXX / 2015, j'ai reçu une réponse d..."


In [3]:
df.shape

(43266, 9)

In [6]:
df.describe()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
count,43266,43266,43266,43266,20760,43266,43266,35568,43266
unique,43266,920,18,150,10,930,5,2,43022
top,Tr-14981,1/19/2017,Debt collection,Incorrect information on credit report,Company has responded to the consumer and the ...,1/19/2017,Closed with explanation,No,"Equifax mishandled my information, which has l..."
freq,1,226,10422,4466,10928,182,34300,27663,13


In [7]:
df.isnull().sum()

Complaint-ID                      0
Date-received                     0
Transaction-Type                  0
Complaint-reason                  0
Company-response              22506
Date-sent-to-company              0
Complaint-Status                  0
Consumer-disputes              7698
Consumer-complaint-summary        0
dtype: int64

In [8]:
# df.dropna(inplace=True)

In [9]:
df.shape

(16961, 9)

In [4]:
df.drop(columns=['Date-received', 'Date-sent-to-company','Complaint-ID'], axis=1, inplace=True)

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prasads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prasads\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prasads\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import re
from string import punctuation
from nltk.corpus import stopwords
stop_list = stopwords.words('english')

In [7]:
from nltk.stem.snowball import SnowballStemmer
s1= SnowballStemmer("english")
s2= SnowballStemmer("french")
s3= SnowballStemmer("german")
s4= SnowballStemmer("portuguese")
s5= SnowballStemmer("russian")
s6= SnowballStemmer("spanish")
s7= SnowballStemmer("swedish")

In [14]:
 #   languages = (
        "arabic",
        "danish",
        "dutch",
        "english",
        "finnish",
        "french",
        "german",
        "hungarian",
        "italian",
        "norwegian",
        "porter",
        "portuguese",
        "romanian",
        "russian",
        "spanish",
        "swedish",)

IndentationError: unexpected indent (<ipython-input-14-2210aec6f277>, line 2)

In [15]:
import numpy as np   
stemmer=[s1,s2,s3,s4,s5,s6,s7]
x=['advertisement','advertising','requête','que','únicamente']
for i in np.arange(0,7,1):
  stems = [stemmer[i].stem(t) for t in x]
stems

['advertisement', 'advertising', 'requêt', 'que', 'únicament']

In [8]:
#text=df['Consumer-complaint-summary']
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = nltk.word_tokenize(text)
    tokens = [' '.join(w for w in tokens if w.lower() not in (stop_list + list(punctuation)))]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stemmer=[s1,s2,s3,s4,s5,s6,s7]
    for i in np.arange(0,7,1):
      stems = [stemmer[i].stem(t) for t in filtered_tokens]
    return " ".join(map(str, stems))

In [9]:
# tokenizedArticles = pd.DataFrame()
df['cleaned_tokens'] = df['Consumer-complaint-summary'].apply(lambda x: tokenize_and_stem(x))
df['word_count'] = df['cleaned_tokens'].apply(lambda x: len(x))

In [14]:
df.head()

,Transaction-Type,Complaint-reason,Complaint-Status,cleaned_tokens
0,Mortgage,"Loan servicing, payments, escrow account",Closed with explanation,seterus inc déposé un faux rapport auprès des ...
1,Credit reporting,Incorrect information on credit report,Closed with non-monetary relief,xx xx xxxx la requête en faillite n ° xxxx du ...
2,Bank account or service,Using a debit or ATM card,Closed with explanation,el xxxx xxxx 15 estaba preparando el vuelo de ...
3,Debt collection,Cont'd attempts collect debt not owed,Closed with explanation,loan paid xxxx xxxx xxxx 4 years moved va move...
4,Credit card,Payoff process,Closed with explanation,j'ai obtenu un compte de crédit de soins pour ...


In [13]:
df.drop(columns=['Consumer-complaint-summary','word_count'], axis=1, inplace=True)

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#df['Company-response'] = le.fit_transform(df['Company-response'])
#df['Consumer-disputes'] = le.fit_transform(df['Consumer-disputes'])
df['Complaint-reason'] = le.fit_transform(df['Complaint-reason'])
df['Transaction-Type'] = le.fit_transform(df['Transaction-Type'])

In [16]:
y=df['Complaint-Status']

In [17]:
df.drop(columns=['Complaint-Status'], axis=1, inplace=True)

In [18]:
df.head()

,Transaction-Type,Complaint-reason,cleaned_tokens
0,10,78,seterus inc déposé un faux rapport auprès des ...
1,5,71,xx xx xxxx la requête en faillite n ° xxxx du ...
2,0,145,el xxxx xxxx 15 estaba preparando el vuelo de ...
3,7,36,loan paid xxxx xxxx xxxx 4 years moved va move...
4,3,100,j'ai obtenu un compte de crédit de soins pour ...


In [19]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, max_features=200000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_tokens'])


print(tfidf_matrix.shape)

Wall time: 51 s
(43266, 59)


In [20]:
tfidf_df = pd.DataFrame(tfidf_matrix.A, columns=tfidf_vectorizer.get_feature_names(),index=df.index)
tfidf_df.head(15)

,00,account,ai,asked,bank,called,card,company,credit,credit report,...,time,told,una,une,xx,xx xx,xx xx xxxx,xx xxxx,xxxx xxxx xxxx,years
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.213506,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.093296,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.651814,0.330917,0.363020,0.361756,0.270891,0.000000
2,0.055973,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.124335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.050641,0.211397,0.000000,0.072526,0.000000,0.000000,0.000000,0.064414,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.117935,0.059874,0.065682,0.065454,0.343092,0.372540
4,0.000000,0.000000,0.597075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.112280,0.000000,0.000000,0.000000,0.000000,0.037861,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.153109,0.000000,0.154676,0.000000,0.000000,...,0.150971,0.149939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.357831
6,0.000000,0.000000,0.126968,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.043773,0.710325,0.252436,0.276925,0.512499,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220066,0.095724,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.216005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.263136,0.000000,0.000000,0.000000,0.000000,0.000000,0.127388,0.000000
9,0.000000,0.000000,0.339365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.350997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
df.drop(columns=['cleaned_tokens'], axis=1, inplace=True)

In [22]:
df.head()

,Transaction-Type,Complaint-reason
0,10,78
1,5,71
2,0,145
3,7,36
4,3,100


In [23]:
df.index

RangeIndex(start=0, stop=43266, step=1)

In [24]:
df_2=df.join(tfidf_df)
df_2.head()

,Transaction-Type,Complaint-reason,00,account,ai,asked,bank,called,card,company,...,time,told,una,une,xx,xx xx,xx xx xxxx,xx xxxx,xxxx xxxx xxxx,years
0,10,78,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.213506,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,5,71,0.093296,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.651814,0.330917,0.363020,0.361756,0.270891,0.00000
2,0,145,0.055973,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.124335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,7,36,0.050641,0.211397,0.000000,0.072526,0.0,0.0,0.0,0.064414,...,0.0,0.0,0.000000,0.000000,0.117935,0.059874,0.065682,0.065454,0.343092,0.37254
4,3,100,0.000000,0.000000,0.597075,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.112280,0.000000,0.000000,0.000000,0.000000,0.037861,0.00000


In [25]:
X=df_2

In [27]:
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from imblearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.datasets import make_classification

In [28]:
 X, y = make_classification(n_classes=4, class_sep=2,
                           weights=[0.2,0.0.3,0.2,0.3], n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)
print('Original dataset shape %s' % Counter(y))

SyntaxError: invalid syntax (<ipython-input-28-3366b803b190>, line 2)

In [0]:
ros = RandomOverSampler(random_state=777)
X_ROS, y_ROS = ros.fit_sample(X, y)                            

In [29]:
smt = SMOTE(random_state=42, k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(X, y) 

In [0]:
ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X, y)

In [31]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(X, y)

#plot_2d_space(X_smt, y_smt, 'SMOTE + Tomek links')

In [0]:
# undersampling
# nm3 = NearMiss(ratio='not minority',random_state=777, version=3, n_neighbors_ver3=4)
# X_nm, y_nm = nm.fit_sample(X,y)

In [0]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('tfidf_df',Pipeline([
        pd.DataFrame(tfidf.A, columns=tfidf_vectorizer.get_feature_names(),index=df.index),
        df.drop(columns=['cleaned_tokens','word_count','Complaint-Status'], axis=1, inplace=True),
        'df_2',df.join(tfidf_df)])
    ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True)))])       # (‘clf’, OneVsRestClassifier(LinearSVC(class_weight=”balanced”)))

parameters = {
    'tfidf__max_df': np.arange(0.5,0.8,0.05),
    'tfidf__min_df': np.arange(0.1,0.4,0.05),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3),
    'clf__class_prior ' : [None, [.1,.9],[.2, .8]]
}

grid_search= GridSearchCV(pipeline, parameters)
grid_search.fit(X_train, y_train)

print("Best parameters set:")
print grid_search.best_estimator_.steps

In [0]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(class_weight='balanced', multi_class='multinomial')
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
print(metrics.classification_report(y_test, pred))

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_SMOTE, y_SMOTE, test_size=0.33, random_state=42)

In [36]:
clf = MultinomialNB()

In [37]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
cm

accuracy:   0.498


array([[2301,  773,  479,  517],
       [1262, 1047,  937,  977],
       [ 435,  413, 2981,  336],
       [ 827,  579,  793, 1936]], dtype=int64)

In [38]:
print(metrics.classification_report(y_test, pred))

                                 precision    recall  f1-score   support

                         Closed       0.48      0.57      0.52      4070
        Closed with explanation       0.37      0.25      0.30      4223
    Closed with monetary relief       0.57      0.72      0.64      4165
Closed with non-monetary relief       0.51      0.47      0.49      4135

                      micro avg       0.50      0.50      0.50     16593
                      macro avg       0.48      0.50      0.49     16593
                   weighted avg       0.48      0.50      0.48     16593



In [39]:
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
grid_search = GridSearchCV(SVC(kernel='linear',class_weight='balanced'), param_grid)
grid_search.fit(X_train, y_train)
grid_search.best_params_

f:\envs\env1\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "f:\envs\env1\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-f18dbfadc277>", line 5, in <module>
    grid_search.fit(X_train, y_train)
  File "f:\envs\env1\lib\site-packages\sklearn\model_selection\_search.py", line 722, in fit
    self._run_search(evaluate_candidates)
  File "f:\envs\env1\lib\site-packages\sklearn\model_selection\_search.py", line 1191, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "f:\envs\env1\lib\site-packages\sklearn\model_selection\_search.py", line 711, in evaluate_candidates
    cv.split(X, y, groups)))
  File "f:\envs\env1\lib\site-packages\sklearn\externals\joblib\parallel.py", line 917, in __call__
    if self.dispatch_one_batch(iterator):
  File "f:\envs\env1\lib\site-packages\sklearn\externals\joblib\parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  Fil

KeyboardInterrupt: 

In [ ]:
grid_search.best_score_              # c=0.1 gamma = 0.001

In [48]:
svc=SVC(kernel='linear',class_weight='balanced',C=0.1,gamma=0.001)
svc.fit(X_train,y_train)
svc_pred= svc.predict(X_test)
print(metrics.classification_report(y_test, pred))

                                 precision    recall  f1-score   support

                         Closed       0.00      0.00      0.00       105
        Closed with explanation       0.75      0.97      0.85      4164
    Closed with monetary relief       0.25      0.12      0.17       482
Closed with non-monetary relief       0.00      0.00      0.00       847

                      micro avg       0.73      0.73      0.73      5598
                      macro avg       0.25      0.27      0.25      5598
                   weighted avg       0.58      0.73      0.64      5598



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
grid_search = GridSearchCV(SVC(kernel='rbf',class_weight='balanced'), param_grid)
grid_search.fit(X_train, y_train)
grid_search.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'C': 10, 'gamma': 1}

In [75]:
grid_search.best_score_

0.6717416175305817

In [47]:
svc_rbf=SVC(kernel='rbf',class_weight='balanced',C=10,gamma=1)
svc_rbf.fit(X_train,y_train)
svc_pred= svc_rbf.predict(X_test)
print( metrics.accuracy_score(y_test,svm_pred))
print(metrics.classification_report(y_test, pred))

NameError: ignored

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(random_state=0, n_jobs=-1, class_weight='balanced')   # {class_label: weight}
model = clf1.fit(X_train, y_train)
predict = model.predict(X_test)
proba = model.predict_proba(X_test)
cm = metrics.confusion_matrix(y_test, predict)
print(metrics.classification_report(y_test, predict))

f:\envs\env1\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


                                 precision    recall  f1-score   support

                         Closed       0.98      0.99      0.98     11205
        Closed with explanation       0.85      0.83      0.84     11374
    Closed with monetary relief       0.93      0.96      0.95     11354
Closed with non-monetary relief       0.90      0.89      0.89     11331
              Untimely response       1.00      0.99      1.00     11331

                      micro avg       0.93      0.93      0.93     56595
                      macro avg       0.93      0.93      0.93     56595
                   weighted avg       0.93      0.93      0.93     56595



In [107]:
X_train

array([[5.00000000e+00, 4.30000000e+01, 4.00000000e+00, ...,
        0.00000000e+00, 2.48111542e-02, 0.00000000e+00],
       [1.00000000e+01, 3.40000000e+01, 3.74986875e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e+00, 2.40000000e+01, 4.00000000e+00, ...,
        0.00000000e+00, 6.99948693e-03, 0.00000000e+00],
       ...,
       [6.00000000e+00, 5.30000000e+01, 7.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e+00, 3.70000000e+01, 9.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.00000000e+00, 3.10000000e+01, 9.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [34]:
print( metrics.accuracy_score(y_test,predict))

0.9313190211149395


In [35]:
cm

array([[11058,   130,     5,    12,     0],
       [  162,  9441,   654,  1101,    16],
       [    6,   394, 10922,    31,     1],
       [   51,  1108,   131, 10036,     5],
       [    4,    73,     0,     3, 11251]], dtype=int64)

In [52]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
scale_pos_weight = [3,4,5]
for i in scale_pos_weight:
    print('scale_pos_weight = {}: '.format(i))
    clf2 = XGBClassifier(scale_pos_weight=i)
    clf2.fit(X_train, y_train)
    predict = clf2.predict(X_test)
    proba = clf2.predict_proba(X_test)
    cm = metrics.confusion_matrix(y_test, predict)
    print(metrics.accuracy_score(y_test, predict))
    print('Confusion Matrix: \n', cm)
    print('--------------------')
    print(metrics.classification_report(y_test, predict))
    print('--------------------')
    print('No : %', round(100 * (cm[0][0]/(cm[0][0]+cm[0][1]))))
    print('Yes : %', round(100* (cm[1][1]/(cm[1][0]+cm[1][1]))))
    print('---------------------------------------------------------')

scale_pos_weight = 3: 
0.6909540167540529
Confusion Matrix: 
 [[3538  189  268   75]
 [ 621 2157  628  817]
 [ 193  291 3552  129]
 [ 732  581  604 2218]]
--------------------
                                 precision    recall  f1-score   support

                         Closed       0.70      0.87      0.77      4070
        Closed with explanation       0.67      0.51      0.58      4223
    Closed with monetary relief       0.70      0.85      0.77      4165
Closed with non-monetary relief       0.68      0.54      0.60      4135

                      micro avg       0.69      0.69      0.69     16593
                      macro avg       0.69      0.69      0.68     16593
                   weighted avg       0.69      0.69      0.68     16593

--------------------
No : % 95.0
Yes : % 78.0
---------------------------------------------------------
scale_pos_weight = 4: 
0.6909540167540529
Confusion Matrix: 
 [[3538  189  268   75]
 [ 621 2157  628  817]
 [ 193  291 3552  129]
 [

In [0]:
maxdf = np.arange(0.5,0.8,0.05)
mindf = np.arange(0.1,0.4,0.05)
param_grid = {'max_df': maxdf, 'min_df' : mindf}
grid_search = GridSearchCV((TfidfVectorizer(ngram_range=(1,3)).fit_transform(df['cleaned_tokens'])), param_grid)

# Testing starts here

In [36]:
df1 = pd.read_csv('test.csv')
df1.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,NaN,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf..."
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,NaN,6/26/2017,NaN,"I submitted a request to XXXX, which is my cur..."
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...


In [59]:
# df1.dropna(inplace=True)

In [37]:
df1.drop(columns=['Date-received', 'Date-sent-to-company','Complaint-ID'], axis=1, inplace=True)

In [38]:
df1['cleaned_tokens'] = df1['Consumer-complaint-summary'].apply(lambda x: tokenize_and_stem(x))
df1['word_count'] = df1['cleaned_tokens'].apply(lambda x: len(x))

In [39]:
df1.drop(columns=['Consumer-complaint-summary','word_count', 'Company-response', 'Consumer-disputes'], axis=1, inplace=True)

In [40]:
le2 = preprocessing.LabelEncoder()
#df1['Company-response'] = le2.fit_transform(df1['Company-response'])
#df1['Consumer-disputes'] = le2.fit_transform(df1['Consumer-disputes'])
df1['Complaint-reason'] = le2.fit_transform(df1['Complaint-reason'])
df1['Transaction-Type'] = le2.fit_transform(df1['Transaction-Type'])

In [41]:
tfidf_vectorizer1 = TfidfVectorizer(max_df=0.5, max_features=200000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, ngram_range=(1,3))

%time tfidf_matrix1 = tfidf_vectorizer1.fit_transform(df1['cleaned_tokens'])


print(tfidf_matrix1.shape)

Wall time: 20.8 s
(18543, 61)


In [42]:
tfidf_df1 = pd.DataFrame(tfidf_matrix1.A, columns=tfidf_vectorizer1.get_feature_names(),index=df1.index)
#tfidf_df1 = tfidf_df1.drop(columns=['es', 'make'], axis=1)

In [43]:
a = set(tfidf_df1.columns)

In [44]:
b = set(tfidf_vectorizer.get_feature_names())

In [45]:
if a == b:
    print('true')

In [46]:
predict1 = model.predict(tfidf_df1)

In [89]:
for x in tfidf_vectorizer1.get_feature_names(): 
    print(x, ' ', end='')
print('')
for y in tfidf_vectorizer.get_feature_names():
    print(y, ' ', end='')

00  account  ai  asked  bank  called  card  company  credit  credit report  date  days  debt  des  el  en  es  et  help  information  je  la  le  les  letter  loan  los  make  mi  mon  money  mortgage  ne  number  paid  para  pas  pay  payment  payments  phone  por  pour  que  received  report  request  said  se  sent  service  time  told  una  une  xx  xx xx  xx xx xxxx  xx xxxx  xxxx xxxx xxxx  years  
00  account  ai  asked  bank  called  card  company  credit  credit report  date  days  debt  des  el  en  et  help  information  je  la  le  les  letter  loan  los  mi  mon  money  mortgage  ne  number  paid  para  pas  pay  payment  payments  phone  por  pour  que  received  report  request  said  se  sent  service  time  told  una  une  xx  xx xx  xx xx xxxx  xx xxxx  xxxx xxxx xxxx  years  

In [48]:
df1 = pd.read_csv('test.csv')
df1.drop(columns=['Date-received', 'Transaction-Type', 'Complaint-reason', 'Company-response', 'Date-sent-to-company', 'Consumer-disputes', 'Consumer-complaint-summary'], axis=1, inplace=True)
df1['Complaint-status'] = predict1
df1.head()

,Complaint-ID,Complaint-status
0,Te-1,Closed with explanation
1,Te-2,Closed with explanation
2,Te-3,Closed with explanation
3,Te-4,Closed with explanation
4,Te-5,Closed with explanation


In [54]:
dir(df1.groupby(['Complaint-status']))

['__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_accessors',
 '_add_numeric_operations',
 '_agg_doc',
 '_aggregate',
 '_aggregate_generic',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_apply_whitelist',
 '_assure_grouper',
 '_block_agg_axis',
 '_bool_agg',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_decide_output_index',
 '_def_str',
 '_define_paths',
 '_deprecations',
 '_dir_additio

In [55]:
df1.groupby(['Complaint-status']).count()

,Complaint-ID
Complaint-status,
Closed,816
Closed with explanation,15353
Closed with monetary relief,940
Closed with non-monetary relief,1432
Untimely response,2
